Name: 
- Mayank Khanna ( CWID:)
- Patrick Simmon ( CWID:)

Note: We Will be attempting the full 20 bonus points that are provided in this assignment.

<H1>Total Attempt: 10 + 20  = 30</H1>

<h1>3. Intent Classificantion using SVM(CS 583 Fall 2021)</h1>
<h2> DataSet used : <a href="https://github.com/clinc/oos-eval">CLINC150 </a>

<h2> Objective: </h2>
<ul>
  <li> Use text data to classify (map) text to a particular intent. This dataset contains 150 intents in total and our objective requires us to map the text to a particular intent. </li>
  <li> This generally finds applications in chatbots where a query from a user is routed to a particular customer service team. If this chatbot is not a part of a ciritcal application, then both false postitive and false negitives are acceptable. However, while building any model, we should strive to reduce false positives and negitives to a minimum. We should keep in mind that an incorrect classification from a model can hurt customer satisfaction for a company.</li>
  <li> For the purpose of this assignment, we will test out model on accuracy, since the applications are low priority and the daataset is almost balanced. </li>
</ul>


<h3> Data assumptions and structure. </h3>

The data file is called 'data_full.json' and is stored in the same directory as this notebook. The structure of the dataset is as follows:
```
Data Format
{

  "train": List of training samples with each sample representing a tuple: (query,class)
          [
               ["query / sentence","intent_class"],
               ["query / sentence","intent_class"],
          ],
  "val":  List of validation samples with each sample representing a tuple: (query,class)
          [
               ["query / sentence","intent_class"],
               ["query / sentence","intent_class"],
          ],
   "test":  List of testing samples with each sample representing a tuple: (query,class)
          [
               ["query / sentence","intent_class"],
               ["query / sentence","intent_class"],
          ],
    ...further similar 3 sets for out-of-scope queries for advance classification.
}
```
  

<ul>
  <li> Since we are given a choice to either test accuracy on the 150 intents that are provided OR the recall on oos queries, we have decided to proceed with the accuracy of intents on the dataset provided. </li>
</ul>


<h3>Approach</h3>
<ul>
<li> We first import the data and clean it, featurize it and build models on it. Follow along the notebook to understand our approach!</li>
</ul>

In [ ]:
import pandas as pd
import json
import re

In [ ]:
with open('data_full.json') as json_file: 
    data_dict = json.load(json_file)

In [ ]:
#Since the train, val and test datasets are already divided, we just directly load them
train_data = data_dict['train']
val_data = data_dict['val']
test_data = data_dict['test']

train_df = pd.DataFrame(train_data, columns =['query', 'intent'])
val_df = pd.DataFrame(val_data, columns =['query', 'intent'])
test_df = pd.DataFrame(test_data, columns =['query', 'intent'])

<h4>Check the percentage of target values in train, val and test to see if the dataset is imbalanced or not.</h4>

In [ ]:
#train data
list(train_df['intent'].value_counts(normalize=True) * 100)
train_df['intent'].value_counts(normalize=True) * 100

schedule_maintenance    0.666667
cancel                  0.666667
share_location          0.666667
goodbye                 0.666667
flight_status           0.666667
                          ...   
calories                0.666667
reminder                0.666667
oil_change_how          0.666667
spending_history        0.666667
sync_device             0.666667
Name: intent, Length: 150, dtype: float64

In [ ]:
#test data
list(test_df['intent'].value_counts(normalize=True) * 100)
test_df['intent'].value_counts(normalize=True) * 100

what_can_i_ask_you      0.666667
carry_on                0.666667
what_is_your_name       0.666667
report_lost_card        0.666667
shopping_list_update    0.666667
                          ...   
update_playlist         0.666667
account_blocked         0.666667
taxes                   0.666667
lost_luggage            0.666667
how_old_are_you         0.666667
Name: intent, Length: 150, dtype: float64

In [ ]:
#Val data
list(val_df['intent'].value_counts(normalize=True) * 100)
val_df['intent'].value_counts(normalize=True) * 100

what_can_i_ask_you      0.666667
carry_on                0.666667
what_is_your_name       0.666667
report_lost_card        0.666667
shopping_list_update    0.666667
                          ...   
update_playlist         0.666667
account_blocked         0.666667
taxes                   0.666667
lost_luggage            0.666667
how_old_are_you         0.666667
Name: intent, Length: 150, dtype: float64

<h4>We observe that all the target values are equally balanced in train, val, test!</h4>

<h3>Number of Datapoints in train, val, test</h3>

In [ ]:
visited = []
cnt = 0
for i in range(0, len(train_df['intent'])):
    if train_df['intent'][i] not in visited: 
        visited.append(train_df['intent'][i])
        cnt += 1

print("Number of data points in train data", train_df.shape)
print('-'*50)
print("The attributes of data :", train_df.columns.values)
print('-'*50)
print("The number of intents in train dataset is",cnt)

Number of data points in train data (15000, 2)
--------------------------------------------------
The attributes of data : ['query' 'intent']
--------------------------------------------------
The number of intents in train dataset is 150


In [ ]:
visited = []
cnt = 0
for i in range(0, len(test_df['intent'])):
    if test_df['intent'][i] not in visited: 
        visited.append(test_df['intent'][i])
        cnt += 1

print("Number of data points in test data", test_df.shape)
print('-'*50)
print("The attributes of data :", test_df.columns.values)
print('-'*50)
print("The number of intents in test dataset is",cnt)

Number of data points in test data (4500, 2)
--------------------------------------------------
The attributes of data : ['query' 'intent']
--------------------------------------------------
The number of intents in test dataset is 150


In [ ]:
visited = []
cnt = 0
for i in range(0, len(val_df['intent'])):
    if val_df['intent'][i] not in visited: 
        visited.append(val_df['intent'][i])
        cnt += 1

print("Number of data points in val data", val_df.shape)
print('-'*50)
print("The attributes of data :", val_df.columns.values)
print('-'*50)
print("The number of intents in val dataset is",cnt)

Number of data points in val data (3000, 2)
--------------------------------------------------
The attributes of data : ['query' 'intent']
--------------------------------------------------
The number of intents in val dataset is 150


In [ ]:
## Printing some random query points from the train dataset
print(train_df['query'].values[0])
print("="*50)
print(train_df['query'].values[3000])
print("="*50)
print(train_df['query'].values[6000])
print("="*50)
print(train_df['query'].values[9000])
print("="*50)
print(train_df['query'].values[12099])
print("="*50)

what expression would i use to say i love you if i were an italian
estimated time to airport from current location, la
if i fly american to los angeles, how many carry ons am i allowed
who is your employeer
do you have a name i should call you by


<h2>We will now begin with data cleaning</h2>

<h4>We see in the dataset that a lot of contractions are present. Contractions are words like What're / What's /I'm that can be expanded into words like What are/ what is / I am. For good data cleaning, we first take care of all these contractions and expand them to their full forms</h4>

In [ ]:
# Reference : https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python 
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    return phrase


<h4>We now remove stopwords. Stopwords are very common words in the english language that don't add any value or meaning to the sentence.</h4>

In [ ]:
#stopword removal
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain']

In [ ]:
preprocessed_query_train = []
# tqdm is for printing the status bar
for sentance in train_df['query'].values:
    sent = decontracted(sentance)
    #Replace characters with \ http://texthandler.com/info/remove-line-breaks-python/
    #sent = sent.replace('\\r', ' ')
    #sent = sent.replace('\\"', ' ')
    #sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)#train_df['intent'].values[i]
    # remove spacial character: https://stackoverflow.com/a/5843547/4084039
    #make everything lower and strip excess spaces
    preprocessed_query_train.append(sent.lower().strip())

In [ ]:
train_df.drop(['query'],axis = 1,inplace = True)
train_df['query'] = preprocessed_query_train

In [ ]:
preprocessed_query_test = []
# tqdm is for printing the status bar
for sentance in test_df['query'].values:
    sent = decontracted(sentance)
    #Replace characters with \ http://texthandler.com/info/remove-line-breaks-python/
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    # remove spacial character: https://stackoverflow.com/a/5843547/4084039
    #make everything lower and strip excess spaces
    preprocessed_query_test.append(sent.lower().strip())

In [ ]:
test_df.drop(['query'],axis = 1,inplace = True)
test_df['query'] = preprocessed_query_test

In [ ]:
preprocessed_query_val = []
# tqdm is for printing the status bar
for sentance in val_df['query'].values:
    sent = decontracted(sentance)
    #Replace characters with \ http://texthandler.com/info/remove-line-breaks-python/
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    # remove spacial character: https://stackoverflow.com/a/5843547/4084039
    #make everything lower and strip excess spaces
    preprocessed_query_val.append(sent.lower().strip())

In [ ]:
val_df.drop(['query'],axis = 1,inplace = True)
val_df['query'] = preprocessed_query_val

In [ ]:
## Printing some random query points from the train dataset

print(train_df['query'].values[0])
print("="*50)
print(train_df['query'].values[3000])
print("="*50)
print(train_df['query'].values[6000])
print("="*50)
print(train_df['query'].values[9000])
print("="*50)
print(train_df['query'].values[12099])
print("="*50)

expression would use say love italian
estimated time airport current location la
fly american los angeles many carry ons am allowed
employeer
name call


<h4> Looking at the sentences above, we see that a lot of words were removed from the query. This can be bad because if the query consists of only stopwords, than the entire sentence will be removed. Let's check if this happened!</h4>

In [ ]:
#Check after cleaning, any query was removed or replaced with an empty string
print("" in preprocessed_query_train)

True


<h4>As expected, the queries were deleted entirely after cleaning. Let's investigate this further</h4>

In [ ]:
#Find the indices where the queries are empty after cleaning.
empty_indices = [i for i,x in enumerate(preprocessed_query_train) if not x]

In [ ]:
original_train_df = pd.DataFrame(train_data, columns =['query', 'intent'])
for i in empty_indices:
  print("Original Query: ",original_train_df['query'].values[i])
  print("Query after cleaning: ",train_df['query'].values[i])
  print("Intent: ",train_df['intent'].values[i])
  print(end = '\n')

Original Query:  where are you from
Query:  
Intent:  where_are_you_from

Original Query:  what all can you do
Query:  
Intent:  what_can_i_ask_you

Original Query:  what can you do
Query:  
Intent:  what_can_i_ask_you

Original Query:  what was that
Query:  
Intent:  repeat

Original Query:  do that
Query:  
Intent:  yes

Original Query:  when will i be off again
Query:  
Intent:  next_holiday

Original Query:  what's up
Query:  
Intent:  greeting

Original Query:  how have you been
Query:  
Intent:  greeting

Original Query:  how are you
Query:  
Intent:  greeting

Original Query:  what's up with you
Query:  
Intent:  greeting

Original Query:  what are you
Query:  
Intent:  are_you_a_bot



<h4> From the code above, we can see that the queries that consist of only stopwords were removed and they all belong to different intents. Had they been sourced from the same intent, we could proceed with this since that would improve the accuracy of the model. However, since the queries belong to different intents here, we will not remove any stopwords</h4>

<h2> Cleaning the data wothout removing any stopwords</h2>

In [ ]:
train_df = pd.DataFrame(train_data, columns =['query', 'intent'])
val_df = pd.DataFrame(val_data, columns =['query', 'intent'])
test_df = pd.DataFrame(test_data, columns =['query', 'intent'])

In [ ]:
preprocessed_query_train = []
# tqdm is for printing the status bar
for sentance in train_df['query'].values:
    sent = decontracted(sentance)
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split())#train_df['intent'].values[i]
    # remove spacial character: https://stackoverflow.com/a/5843547/4084039
    #make everything lower and strip excess spaces
    preprocessed_query_train.append(sent.lower().strip())\


train_df.drop(['query'],axis = 1,inplace = True)
train_df['query'] = preprocessed_query_train

In [ ]:
preprocessed_query_test = []
# tqdm is for printing the status bar
for sentance in test_df['query'].values:
    sent = decontracted(sentance)
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split())
    # remove spacial character: https://stackoverflow.com/a/5843547/4084039
    #make everything lower and strip excess spaces
    preprocessed_query_test.append(sent.lower().strip())

test_df.drop(['query'],axis = 1,inplace = True)
test_df['query'] = preprocessed_query_test

In [ ]:
preprocessed_query_val = []
# tqdm is for printing the status bar
for sentance in val_df['query'].values:
    sent = decontracted(sentance)
    #Replace characters with \ http://texthandler.com/info/remove-line-breaks-python/
    #sent = sent.replace('\\r', ' ')
    #sent = sent.replace('\\"', ' ')
    #sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split())
    # remove spacial character: https://stackoverflow.com/a/5843547/4084039
    #make everything lower and strip excess spaces
    preprocessed_query_val.append(sent.lower().strip())

val_df.drop(['query'],axis = 1,inplace = True)
val_df['query'] = preprocessed_query_val

In [ ]:
# Check if any of the queries were removed after cleaning
print("" in preprocessed_query_train)

False


In [ ]:
print("" in preprocessed_query_test)

False


In [ ]:
print("" in preprocessed_query_val)

False


<h4>Great! It looks like none of the queries were removed!</h4>

<h1>To Do: Try stemming and lemmitization</h1>
<h1>To Do: Data featurization - Bag of Words(unigram, bi gram), Tfidf, W2V, Tfidf weighted W2v</h1>
<h1>To Do: Model Building - SVM(main) with hyperparameter tuning using grid search(need to get accuracy above 88 at any cost), Logistic regression, naive bayes(maybe), Decision trees, random forests, XG boost, Stacking classifier, cascading classifier, Neural Networks</h1>